# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
import os 
import json 
from dotenv import load_dotenv 

from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

# Lets do little different. Grab ollama models to do this activity 

In [3]:
load_dotenv(override=True)

True

In [7]:
api_key=os.getenv("OPENAI_API_KEY")
ollama_api_key='ollama'
llama_model = os.getenv("LLAMA3_1_MODEL")
gemma_model = os.getenv("GEMMA3_MODEL")
local_base_url=os.getenv("OLLAMA_BASE_URL")

In [8]:
llama=OpenAI(base_url=local_base_url, api_key=ollama_api_key)
gemma=OpenAI(base_url=local_base_url, api_key=ollama_api_key)

# Scrape the details from Ed Donner's website

In [9]:
links=fetch_website_links(url="https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

### Step 1 - call Gemma3 model to find which all links are relevant

In [10]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [11]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [12]:
print(get_links_user_prompt(url="https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [51]:
def select_relevant_links(url:str, model:OpenAI=gemma, model_name:str="gemma3"):
    response=model.chat.completions.create(model=model_name, messages=[
        {"role" : "system", "content" : link_system_prompt}, 
        {"role" : "user", "content" : get_links_user_prompt(url)}], response_format={"type" : "json_object"
    })

    result=response.choices[0].message.content
    links=json.loads(result)

    print(f"Found {len(links['links'])} relevant links")

    return links


In [52]:
select_relevant_links(url="https://edwarddonner.com")

Found 14 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'about page', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'careers page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/'}]}

In [53]:
# call this for hugging face 
select_relevant_links(url="https://huggingface.co")

Found 23 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'models', 'url': 'https://huggingface.co/models/'},
  {'type': 'datasets', 'url': 'https://huggingface.co/datasets/'},
  {'type': 'spaces', 'url': 'https://huggingface.co/spaces/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/'},
  {'type': 'documentation',
   'url': 'https://huggingface.co/docs/transformers/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/diffusers/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/safetensors/'},
  {'type': 'documentation',
   'url': 'https://huggingface.co/docs/huggingface_hub/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/tokenizers/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/trl/'},
  {'type': 'documentation',
   'url': 'https://huggingface.co/docs/transformers.js/'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs/smolagents/'},
  {'type': 'documentatio

### Step 2 - Make a brochure

In [54]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [55]:
fetch_page_and_all_relevant_links(url="https://huggingface.co")

Found 17 relevant links


'## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\nabout 17 hours ago\n•\n623k\n•\n1.82k\nPaddlePaddle/PaddleOCR-VL\nUpdated\n1 day ago\n•\n14.8k\n•\n1.07k\ntencent/HunyuanWorld-Mirror\nUpdated\nabout 12 hours ago\n•\n5.94k\n•\n330\nkrea/krea-realtime-video\nUpdated\n5 days ago\n•\n1.04k\n•\n179\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n10 days ago\n•\n262k\n•\n320\nBrowse 1M+ models\nSpaces\nRunning\n514\n514\nveo3.1-fast\n🐨\nGenerate videos from text or images\nRunning\n15.4k\n15.4k\nDeepSite v3\n🐳\nGenerate any application by Vibe Coding\nRunning\non\nZero\n225\n225\nDeepSeek OCR Demo\n🆘\nAn interactive demo for the DeepSeek

In [56]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

In [57]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [58]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found 12 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\nabout 17 hours ago\n•\n623k\n•\n1.82k\nPaddlePaddle/PaddleOCR-VL\nUpdated\n1 day ago\n•\n14.8k\n•\n1.07k\ntencent/HunyuanWorld-Mirror\nUpdated\nabout 12 hours ago\n•\n5.94k\n•\n330\nkrea/krea-realtime-video\nUpdated\n5 days ago\n•\n1.04k\n•\n179\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n10 days ago\n•\n262k\n•\n320\nBrowse 1M+ models\nSpaces\nRunning\n514\n514\

In [60]:
def create_brochure(company_name:str, url:str, model:OpenAI=llama, model_name="llama3.1"):
    response = model.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [61]:
create_brochure(company_name="HuggingFace", url="https://huggingface.co")

Found 20 relevant links


**Hugging Face: The AI Collaboration Platform**

At Hugging Face, we're building the future of artificial intelligence by enabling the community to collaborate on models, datasets, and applications. Our platform is designed to accelerate machine learning development and deployment.

**Our Mission**
----------------

We aim to create a transparent and inclusive environment where researchers, developers, and businesses can work together to advance AI. By providing open-source tools and resources, we empower our community to build innovative solutions that benefit humanity.

**Key Features**

* **Open-Source Stack**: Our collection of open-source models, datasets, and libraries enables faster development and collaboration.
* **Unlimited Public Models & Datasets**: Anyone can create, share, and collaborate on unlimited public models and datasets.
* **Private Compute & Enterprise Solutions**: Paid options for enterprise-grade security, access controls, and dedicated support.

**Community**
-------------

Hugging Face is home to the machine learning community. We host over 1 million models and hundreds of thousands of datasets. Our platform allows you to create a portfolio, share your work with the world, and build your AI profile.

**Who We Serve**
----------------

* **Researchers**: Accelerate your ML research with our open-source stack.
* **Developers**: Collaborate on models and applications using our platform.
* **Businesses**: Leverage private compute and enterprise solutions for security and support.

**Join Our Community**
----------------------

Sign up today to explore AI apps, browse millions of models and datasets, and collaborate with the Hugging Face community. Accelerate your machine learning journey with us!

# Let's do typewrite animation i.e. stream the data which is created by LLM and show it on realtime

In [65]:
def stream_brochure(company_name, url, model:OpenAI=llama, model_name="llama3.1"):
    stream=model.chat.completions.create(model=model_name, messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ], stream=True)

    response=""
    display_handle=display(Markdown(""), display_id=True)

    for chunk in stream:
        response+=chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)

In [66]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found 12 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


**Welcome to Hugging Face**
===============

Hugging Face is a collaboration platform and open-source software that enables machine learning researchers, developers, and businesses to create, share, and use pre-trained models and datasets.

**Community-driven Platform**
-----------------------------

*   **Collaboration**: Host and collaborate on unlimited public models, datasets, and applications.
*   **Open-Source Stack**: Move faster with the HF Open source stack.
*   **Portfolio Building**: Build your portfolio by sharing your work with the world and creating an ML profile.

**Unlock Business Potential**
-----------------------------

*   **Accelerate Your AI Journey**: We provide paid Compute and Enterprise solutions for advanced business operations.
*   **Enterprise-Grade Security and Support**: Give your team access to the most secure, scalable platform with dedicated support.

**Explore Our Solutions**
-------------------------

### Models

Discover our extensive collection of pre-trained models in various libraries, including but not limited to TensorFlow, PyTorch, ONNX, and Transformers. Browse over 1 million+ models.

### Datasets

Access a huge collection of datasets from various sources, facilitating tasks such as text generation, image-to-text, and text-to-image tasks. Browse our dataset repository.

**Get Started Today**
---------------------

Signing up is easy! Click on the "Sign Up" button to begin exploring and collaborating with our platform.